# Problem 1. Mushroom Weight and Height

In the class we build a naive bayes classifier which classify whether a mushroom is poisonous or edible. In the class, all the feature are of categorical type; it has a discrete number of outcome as opposed to real number.

In this problem we want to explore two ways to deal with real number features for Naive Bayes classifier.

## The Data
The data given to you is very similar to mushroom data. It has three features: cap-color(with the same dictionary we did in class), mushroom-weight(made up by me), mushroom-height(also made up by me).

The data is given in
`mushrooms_homework_test.csv`
and
`mushrooms_homework_train.csv`

## Task 1.1) Simplest way. Just bin it.

The simplest way for dealing with continuous value feature is to discretize it. The simplest way to discretize is just to bin it. For example if your data looks like

`data = (0.9, 1.1, 1.2, 2.1, 2.2, 4.2, 5.3, 5.1)`

We count bin it with bin edges of 

`bins = (1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5)`

the bin number of a data point $x$ is the index $i$ where `bins[i-1] < x < bins[i]`. If $x$ is less than the minimum of bin edge then the bin number is 0. If $x$ is more than the maximum of bin edge then it's `len(bins)`.

Ex the data points above would be turned into
`binno = (0, 1, 1, 3, 3, 7, 9, 9)`

Once we discretize the value we can just use Bayes Classifier we did in class.

**Your task is to build a naive bayes classifier with binned height and binned weight. Pick appropriate bin edges somehow.
Then test your algorithm on the test data set. Report how many you got right and wrong in test data**


In [149]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import keyword
from IPython.display import set_matplotlib_formats
import patternrekt as prekt
set_matplotlib_formats('retina')
plt.style.use('seaborn-notebook')

In [150]:
#reading Data (civilized way)
df = pd.read_csv('data\mushrooms_homework_train.csv')
#fix messy column names
#print(df.columns)
df.columns = df.columns \
    .str.strip() \
    .str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('-','_') \
    .map(lambda x: 'x'+x if x in keyword.kwlist else x )

In [151]:
display(df)

,xclass,cap_color,weight,height
0,e,y,6.122458,7.143689
1,e,w,4.709259,7.398728
2,p,w,2.341551,4.733059
3,e,g,3.954025,4.040427
4,e,y,3.456429,6.422466
...,...,...,...,...
6502,p,n,3.103014,3.211495
6503,e,n,5.162033,6.841161
6504,e,n,4.754516,5.347342
6505,e,n,3.272145,8.031833


In [152]:
weight = np.array(df.weight)
height = np.array(df.height)

In [153]:
def bin_data(data):
    bins = np.linspace(np.min(data), np.max(data), 3)
    return np.digitize(data, bins)

print("weight:", weight)
print("height:", height)
print("binned weight:", bin_data(weight))
print("binned height:", bin_data(height))
# np.min(height)

weight: [6.12245815 4.70925948 2.34155084 ... 4.75451599 3.27214478 4.19960714]
height: [7.14368883 7.39872822 4.7330586  ... 5.34734223 8.03183258 4.25034002]
binned weight: [2 1 1 ... 2 1 1]
binned height: [1 1 1 ... 1 1 1]


In [154]:
df['bin_weight'] = bin_data(weight)
df['bin_height'] = bin_data(height)
display(df)

,xclass,cap_color,weight,height,bin_weight,bin_height
0,e,y,6.122458,7.143689,2,1
1,e,w,4.709259,7.398728,1,1
2,p,w,2.341551,4.733059,1,1
3,e,g,3.954025,4.040427,1,1
4,e,y,3.456429,6.422466,1,1
...,...,...,...,...,...,...
6502,p,n,3.103014,3.211495,1,1
6503,e,n,5.162033,6.841161,2,1
6504,e,n,4.754516,5.347342,2,1
6505,e,n,3.272145,8.031833,1,1


In [155]:
xclasses = df['xclass']
features = df[['cap_color','bin_weight','bin_height']]

In [156]:
def max_value_key(prob_dict):
    best_key = None
    best_val = 0.
    first = True
    for k, v in prob_dict.items():
        if first or v > best_val:
            best_key = k
            best_val = v
            first = False
    return best_key

In [157]:
from dataclasses import dataclass

@dataclass(frozen=True)
class ProbKey:
    fname: str
    value: str
    cls: str

class NaiveBayes:
    def __init__(self):
        self.prob_dict = {}
        self.xclasses = []
        self.fnames = []
        self.prior = {}
        
    def _cal_prob(self, xclasses, features, fname, value, cls):
        value_mask = features[fname] == value
        cls_mask = xclasses == cls
        n_right = np.sum(value_mask & cls_mask)
        n_cls = sum(cls_mask)
        return n_right/n_cls
        
        
    def train(self, xclasses, features):
        self.xclasses = xclasses.unique()
        self.fnames = features.columns
        prob_dict = {}
        self.prior = {cls: sum(xclasses == cls)/len(xclasses) for cls in self.xclasses}
        
        for fname in self.fnames:
            for val in features[fname].unique():
                for cls in self.xclasses:
                    key = ProbKey(fname = fname, value = val, cls = cls)
                    prob = self._cal_prob(xclasses, features, fname, val, cls)
                    prob_dict[key] = prob
        self.prob_dict = prob_dict
        
    def predict_top(self, data, cls):
        p = self.prior[cls]
        for fname in self.fnames:
            value = getattr(data, fname)
            key = ProbKey(fname, value, cls)
            p *= self.prob_dict[key]
        return p
    
    def predict_prob(self, data):
        top_dict = {cls: self.predict_top(data, cls) for cls in self.xclasses}
        evidence = sum(v for k, v in top_dict.items())
        return {cls: v/evidence for cls, v, in top_dict.items()}
    
    def best_cls(self, data):
        probs = self.predict_prob(data)
        best_cls = max_value_key(probs)
        return best_cls

In [158]:
nb = NaiveBayes()
nb.train(xclasses, features)

In [159]:
#reading Data (civilized way)
df_test = pd.read_csv('data\mushrooms_homework_test.csv')
#fix messy column names
#print(df.columns)
df_test.columns = df_test.columns \
    .str.strip() \
    .str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('-','_') \
    .map(lambda x: 'x'+x if x in keyword.kwlist else x )

In [160]:
test_weight = np.array(df_test.weight)
test_height = np.array(df_test.height)

In [161]:
df_test['bin_weight'] = bin_data(test_weight)
df_test['bin_height'] = bin_data(test_height)

In [162]:
test_xclasses = df_test['xclass']
test_features = df_test[['cap_color','bin_weight','bin_height']]

In [163]:
correct = 0
total = 0
for data, cls in zip(test_features.itertuples(), test_xclasses):
    if nb.best_cls(data) == cls:
        correct += 1
    total += 1
print('Right:', correct, total, correct/total)
print('Wrong:', total - correct, total, 1 - correct/total)

Right: 1175 1617 0.7266542980828695
Wrong: 442 1617 0.2733457019171305


## Task 1.2) Gaussian Naive Bayes.

We could assume a certain probability distribution function(pdf) for the conditional probability. One popular choice is normal distrubution/gaussian distribution.

Let us understand this assumption intuitively. The idea is that the *weight* of *poisonous* mushroom is normally distributed around some mean with some width while the *weight* for edible mushroom is hopefully normally distributed around some other mean.

Let us say that the weight of poisonous mushroom is normally distributed around $2\pm1$ gram(I made up this number)  while the edible mushroom is normally distributed at $5\pm 1$ gram. If we found a mushroom of 2.5 gram. It's probably the poisonous one since edible one should weight around 5 gram.

<img src="gaussian.png" width="400"/>

Concretely, we assume that the probability distribution of feature $X$ given that it is of class $y$ to be

$$
\displaystyle
pdf(X=x|y) = \frac{1}{2\sqrt{\pi}} \exp\left[\frac{-(x-\mu_y)^2}{2\sigma_y^2}\right]
$$

 - $\mu_y$ is the mean of feature $X$ given that it is of class $y$. Ex: mean weight($X$) of poisonous mushroom (eg: 2 gram).

 - $\sigma_y$ is the standard deviation of feature $X$ given that it is of class $y$. Ex std. dev. of weight of poisonous mushroom(eg: \pm 1 gram)
 
Recall the relatio between pdf and probability.
$$
    P(X \in (x, x+\delta x)|y) = \int^{x+\delta x}_x pdf(X=x) \;\text{d}x
$$

for small enough $\delta x$ it becomes
$$
    P(X \in (x, x+\delta x)|y) = pdf(X=x) \delta x
$$

Now here is the important part. From the above $P(X=x|y)$ and $P(X=x|\sim y)$ has a factor of $\delta x$(I drop of the range for brevity). This means that the factor of $\delta x$ appear in both the numerator and denominator of the formula we use for calculating probability. Thus the $\delta x$ cancels out nicely. This means that **we can just use pdf(X=x|y) as P(X=x|y)** in naive bayes formula we got and every thing will just work out. We don't have to worry about the $\delta x$ part

**Your task: build a classifier similar to what you did in 1.1. Except now you use gaussian distribution assumption for the continous features. Measure your performance against the test data**

In [164]:
from typing import List

def gaussian(x, mu, sigma):
    return (1/(2 * np.sqrt(np.pi)))* np.exp((-(x - mu)**2) / (2 * sigma**2))

In [175]:
@dataclass(frozen=True)
class GaussianParam:
    mu: float
    sigma: float

@dataclass(frozen=True)
class GaussianKey:
    fname: str
    cls: str
        
class GaussianNaiveBayes:
    def __init__(self):
        self.prob_dict = {}
        self.xclasses = []
        self.categorical_feat = []
        self.continuous_feat = []
        self.prior = {}
        self.gaussian_params = {}
        
    def _cal_prob(self, xclasses, features, fname, value, cls):
        value_mask = features[fname] == value
        cls_mask = xclasses == cls
        n_right = np.sum(value_mask & cls_mask)
        n_cls = sum(cls_mask)
        return n_right/n_cls
        
        
    def train(self, xclasses, features, continuous_features):
        if continuous_features is None:
            continuous_features = []
        self.xclasses = xclasses.unique()
        self.categorical_feat = [feat for feat in features.columns if feat not in continuous_features]
        self.continuous_feat = continuous_features
        prob_dict = {}
        self.prior = {cls: sum(xclasses == cls)/len(xclasses) for cls in self.xclasses}
        
        for fname in self.categorical_feat:
            for val in features[fname].unique():
                for cls in self.xclasses:
                    key = ProbKey(fname = fname, value = val, cls = cls)
                    prob = self._cal_prob(xclasses, features, fname, val, cls)
                    prob_dict[key] = prob
        self.prob_dict = prob_dict
        
        def set_gaussian_param(col, cls):
            feature = features[col][xclasses == cls]
            return GaussianParam(mu = np.mean(feature), sigma = np.std(feature))
        
        self.gaussian_params = {GaussianKey(fname, cls): set_gaussian_param(fname, cls)
                                for fname in self.continuous_feat 
                                for cls in self.xclasses}
        
    def predict_top(self, data, cls):
        p = self.prior[cls]
        for fname in self.categorical_feat:
            value = getattr(data, fname)
            key = ProbKey(fname, value, cls)
            p *= self.prob_dict[key]
            
        for fname in self.continuous_feat:
            gp = self.gaussian_params[GaussianKey(fname, cls)]
            value = getattr(data, fname)
            p *= gaussian(value, gp.mu, gp.sigma)
        return p
    
    def predict_prob(self, data):
        top_dict = {cls: self.predict_top(data, cls) for cls in self.xclasses}
        evidence = sum(v for k, v in top_dict.items())
        return {cls: v/evidence for cls, v, in top_dict.items()}
    
    def best_cls(self, data):
        probs = self.predict_prob(data)
        best_cls = max_value_key(probs)
        return best_cls

In [176]:
xclasses = df['xclass']
features = df[['cap_color','weight','height']]
continuous_features = ['weight', 'height']

In [177]:
gnb = GaussianNaiveBayes()
gnb.train(xclasses, features, continuous_features)

In [178]:
test_xclasses = df_test['xclass']
test_features = df_test[['cap_color','weight','height']]

In [179]:
correct = 0
total = 0
for data, cls in zip(test_features.itertuples(), test_xclasses):
    if gnb.best_cls(data) == cls:
        correct += 1
    total += 1
print('Right:', correct, total, correct/total)
print('Wrong:', total - correct, total, 1 - correct/total)

Right: 1195 1617 0.7390228818800247
Wrong: 422 1617 0.26097711811997526


# Problem 2. Product Reviews

Naive Bayes is quite powerful given its simplicity. Typically the usefulness of a Machine learning Algorithm is limited only by your imgination on what to ask. If you ask an interesting question, you got a useful system. If you ask a dump question, you got a useless system.

In this problem we will explore a text mining application using Naive Bayes.

The goal of this problem is to make a system that can read customer review and tells whether the customer recommend the product to others or not.

The data is stolen from https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews
I splitted it into train(`clothing_reviews_train.csv`) and test(`clothing_reviews_test.csv`) for you.

The two columns that is relavant to this problem are.
- Recommended IND
- Title
- Review Text

**Do not use any other column**

You could do challenging version (No extra point except for bragging rights)
Use the data from http://jmcauley.ucsd.edu/data/amazon/ and do similar thing --> the book review is hugeeeeee


## Your task
Build a classifier which you can give it a reviewtext and review title and it can split out whether the reviewer recommend the product or not. Measure the performance on test data `clothing_reviews_test.csv`.

## Some Guide for you.

- Be sure to normalize your text. Example [here](https://machinelearningmastery.com/clean-text-machine-learning-python/). This includes
    - lowercase everything
    - clean out stop words
    - get rid of punctuations
    - stem the word
    - Yes you may use nltk only for cleaning up part.
- Be careful as you are multiplying a whole bunch of small numbers together. You are better off adding the log and exponentiate it back.
- Read up lecture notes on spam filtering. Especially on how to deal with missing words. You can read up [old version of excercise 1 from sam](https://github.com/KongsakTi/PatternReg/tree/master/Exercise%201)
- **Do not** get stuck on this alone. Find help/Consult your friends if you are stuck. Collaboration is allowed but you must understand what you send in.

In [1]:
#you are now on your own!!!